<a href="https://colab.research.google.com/github/halylo/TriTueNhanTao/blob/main/BaoCao_TTNT_Buoi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Mô tả chung
Trong trò chơi Tic-Tac-Toe, thuật toán Minimax được dùng để giúp AI chọn nước đi tối ưu, đảm bảo AI không thua và luôn đạt kết quả tốt nhất có thể. Ý tưởng chính của Minimax là mô phỏng toàn bộ các nước đi có thể xảy ra và đánh giá kết quả cuối cùng (thắng/thua/hòa) nếu đi theo hướng đó.
---

*Ý tưởng chính
Duyệt toàn bộ không gian trạng thái (cây nước đi) từ trạng thái hiện tại đến trạng thái kết thúc.
Mỗi trạng thái kết thúc có một giá trị utility: +1 (X thắng), −1 (O thắng), 0 (hòa).
Có 2 hàm đệ quy: maxValue (khi lượt của người tối đa hoá — X) trả giá trị lớn nhất có thể; minValue (khi lượt của người tối thiểu hoá — O) trả giá trị nhỏ nhất có thể.
minimax thử từng nước đi hợp lệ, gọi Minimax đệ quy để đánh giá kết quả, rồi chọn nước tốt nhất cho người hiện tại



In [6]:
import copy
import math
import random
import numpy


X = "X"
O = "O"
EMPTY = None
user = None
ai = None


def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]


Khởi tạo bàn cờ rỗng 3×3.

In [7]:
def player(board):
    """
    Returns player who has the next turn on a board.
    """
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1

    if count % 2 != 0:
        return ai
    return user

Chức năng: Trả người có lượt tiếp theo (global user hoặc ai).

Input: board (list 2D).

Output: giá trị ai hoặc user (chuỗi "X" hoặc "O" được gán trong __main__).

Cách hoạt động: đếm ô đã được đánh (if j nghĩa là j khác None), nếu số ô là lẻ → lượt tiếp theo là ai; nếu chẵn → user.

In [8]:
def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res

Chức năng: Trả tập các nước đi hợp lệ hiện tại (tọa độ (i,j) ô trống).

Input: board.

Output: set các tuple (row, col).

Lưu ý: Tập trả về không có thứ tự — Minimax thử mọi action.

In [9]:
def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board

Chức năng: Trả trạng thái mới khi thực hiện action trên board.

Input: board, action là (i,j).

Output: bản sao board với ô (i,j) được đặt thành người đang đi.

Lưu ý:

Dùng deepcopy để không thay đổi board gốc (quan trọng cho Minimax).

In [10]:
def get_horizontal_winner(board):
    # check horizontally
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[i][0]
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val

    return winner_val


Chức năng: Kiểm tra từng hàng xem có 3 ô cùng ký tự ("X" hoặc "O") không.

Output: "X" hoặc "O" nếu có hàng thắng; ngược lại None.

Lưu ý:

Khởi winner_val bằng ô đầu hàng; nếu đó là None (ô trống) thì winner_val là None → không trả thắng giả.

Cách so sánh != với None an toàn vì None != "X".

In [11]:
def get_vertical_winner(board):
    # check vertically
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[0][i]
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val

    return winner_val

Chức năng: Kiểm tra từng cột xem có 3 ô cùng ký tự ("X" hoặc "O") không.

Output: "X" hoặc "O" nếu có hàng thắng; ngược lại None.

Lưu ý:

Khởi winner_val bằng ô đầu hàng; nếu đó là None (ô trống) thì winner_val là None → không trả thắng giả.

Cách so sánh != với None an toàn vì None != "X".

In [12]:
def get_diagonal_winner(board):
    # check diagonally
    winner_val = None
    board_len = len(board)

    # main diagonal
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
    if winner_val:
        return winner_val

    # anti-diagonal
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None

    return winner_val


Chức năng: Kiểm tra 2 đường chéo chính (main & anti).

Output: "X" hoặc "O" nếu 3 ô chéo cùng ký tự; else None.

Lưu ý: tương tự vấn đề None nếu ô đầu là trống.

In [13]:
def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    winner_val = (
        get_horizontal_winner(board)
        or get_vertical_winner(board)
        or get_diagonal_winner(board)
        or None
    )
    return winner_val

Chức năng: Gom kết quả từ 3 hàm kiểm tra; trả người thắng đầu tiên tìm được (ưu tiên hàng → cột → chéo).

Output: "X", "O", hoặc None.

In [14]:
def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) is not None:
        return True

    for i in board:
        for j in i:
            if j == EMPTY:
                return False

    return True

Chức năng: Kiểm tra xem game đã kết thúc chưa.

Trả: True nếu có người thắng hoặc không còn ô trống; False nếu vẫn còn ô trống và chưa có người thắng.

In [15]:
def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0

Chức năng: Đánh giá trạng thái kết thúc.

Output: 1 nếu X thắng, -1 nếu O thắng, 0 nếu hòa hoặc chưa có người thắng.

Lưu ý: Hàm chỉ có nghĩa khi board là terminal; nhưng code vẫn gọi trên terminal check trong min/max.

In [16]:
def maxValue(state):
    if terminal(state):
        return utility(state)

    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v


def minValue(state):
    if terminal(state):
        return utility(state)

    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v


Chức năng: Trả giá trị tốt nhất mà người tối đa hoá (X) và tối thiểu hóa (O) có thể đạt từ state.

Cách hoạt động: nếu terminal → trả utility; else lặp qua actions, chọn giá trị lớn nhất trong các giá trị do minValue và giá trị nhỏ nhất trong maxValue trả về.

Lưu ý: Hàm đệ quy, base case là terminal.

In [17]:
def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    current_player = player(board)

    if current_player == X:
        best = -math.inf
        for action in actions(board):
            val = minValue(result(board, action))
            if val > best:
                best = val
                move = action
    else:
        best = math.inf
        for action in actions(board):
            val = maxValue(result(board, action))
            if val < best:
                best = val
                move = action

    return move

Chức năng: Trả action tối ưu cho người đang có lượt.

Cách hoạt động:

Nếu hiện tại là X → chọn action với giá trị minValue lớn nhất (vì X muốn max).

Nếu hiện tại là O → chọn action với maxValue nhỏ nhất (O muốn min).

In [18]:

if __name__ == "__main__":
    board = initial_state()
    ai_turn = False

    print("Choose a player")
    user = input()

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            win = winner(board)
            if win is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {win} wins.")
            break

        else:
            if user != playr and not game_over:
                if ai_turn:
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    print(numpy.array(board))

            elif user == playr and not game_over:
                ai_turn = True
                print("Enter the position to move (row,col)")
                i = int(input("Row: "))
                j = int(input("Col: "))

                if board[i][j] == EMPTY:
                    board = result(board, (i, j))

                print(numpy.array(board))


Choose a player
O
Enter the position to move (row,col)
Row: 1
Col: 1
[[None None None]
 [None 'O' None]
 [None None None]]
[['X' None None]
 [None 'O' None]
 [None None None]]
Enter the position to move (row,col)
Row: 0
Col: 2
[['X' None 'O']
 [None 'O' None]
 [None None None]]
[['X' None 'O']
 [None 'O' None]
 ['X' None None]]
Enter the position to move (row,col)
Row: 1
Col: 0
[['X' None 'O']
 ['O' 'O' None]
 ['X' None None]]
[['X' None 'O']
 ['O' 'O' 'X']
 ['X' None None]]
Enter the position to move (row,col)
Row: 2
Col: 2
[['X' None 'O']
 ['O' 'O' 'X']
 ['X' None 'O']]
[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' None 'O']]
Enter the position to move (row,col)
Row: 1
Col: 1
[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' None 'O']]
Enter the position to move (row,col)
Row: 2
Col: 1
[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' 'O' 'O']]
Game Over: Tie.


Khởi tạo board, hỏi người chơi muốn là "X" hay "O" (gán user, ai).

Vòng while True: kiểm tra terminal(board); nếu game over thì in kết quả và break.

Nếu chưa kết thúc:

Nếu đến lượt AI (user != playr) và ai_turn true → AI gọi minimax và thực hiện nước.

Nếu đến lượt user → đọc input Row/Col, nếu hợp lệ và ô trống thì đặt nước bằng result.

ai_turn là cờ bổ trợ để tránh AI đi hai lần liên tiếp trong logic hiện tại.

In [5]:
import copy
import math
import random
import numpy


X = "X"
O = "O"
EMPTY = None
user = None
ai = None


def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]


def player(board):
    """
    Returns player who has the next turn on a board.
    """
    count = 0
    for i in board:
        for j in i:
            if j:
                count += 1

    if count % 2 != 0:
        return ai
    return user


def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    res = set()
    board_len = len(board)
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY:
                res.add((i, j))
    return res


def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    curr_player = player(board)
    result_board = copy.deepcopy(board)
    (i, j) = action
    result_board[i][j] = curr_player
    return result_board


def get_horizontal_winner(board):
    # check horizontally
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[i][0]
        for j in range(board_len):
            if board[i][j] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val

    return winner_val


def get_vertical_winner(board):
    # check vertically
    winner_val = None
    board_len = len(board)

    for i in range(board_len):
        winner_val = board[0][i]
        for j in range(board_len):
            if board[j][i] != winner_val:
                winner_val = None
        if winner_val:
            return winner_val

    return winner_val


def get_diagonal_winner(board):
    # check diagonally
    winner_val = None
    board_len = len(board)

    # main diagonal
    winner_val = board[0][0]
    for i in range(board_len):
        if board[i][i] != winner_val:
            winner_val = None
    if winner_val:
        return winner_val

    # anti-diagonal
    winner_val = board[0][board_len - 1]
    for i in range(board_len):
        j = board_len - 1 - i
        if board[i][j] != winner_val:
            winner_val = None

    return winner_val


def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    winner_val = (
        get_horizontal_winner(board)
        or get_vertical_winner(board)
        or get_diagonal_winner(board)
        or None
    )
    return winner_val


def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) is not None:
        return True

    for i in board:
        for j in i:
            if j == EMPTY:
                return False

    return True


def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1
    elif winner_val == O:
        return -1
    return 0


def maxValue(state):
    if terminal(state):
        return utility(state)

    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v


def minValue(state):
    if terminal(state):
        return utility(state)

    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v


def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    current_player = player(board)

    if current_player == X:
        best = -math.inf
        for action in actions(board):
            val = minValue(result(board, action))
            if val > best:
                best = val
                move = action
    else:
        best = math.inf
        for action in actions(board):
            val = maxValue(result(board, action))
            if val < best:
                best = val
                move = action

    return move


if __name__ == "__main__":
    board = initial_state()
    ai_turn = False

    print("Choose a player")
    user = input()

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            win = winner(board)
            if win is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {win} wins.")
            break

        else:
            if user != playr and not game_over:
                if ai_turn:
                    move = minimax(board)
                    board = result(board, move)
                    ai_turn = False
                    print(numpy.array(board))

            elif user == playr and not game_over:
                ai_turn = True
                print("Enter the position to move (row,col)")
                i = int(input("Row: "))
                j = int(input("Col: "))

                if board[i][j] == EMPTY:
                    board = result(board, (i, j))

                print(numpy.array(board))


Choose a player
O
Enter the position to move (row,col)
Row: 1
Col: 1
[[None None None]
 [None 'O' None]
 [None None None]]
[['X' None None]
 [None 'O' None]
 [None None None]]
Enter the position to move (row,col)
Row: 2
Col: 2
[['X' None None]
 [None 'O' None]
 [None None 'O']]
[['X' None None]
 [None 'O' None]
 ['X' None 'O']]
Enter the position to move (row,col)
Row: 2
Col: 0
[['X' None None]
 [None 'O' None]
 ['X' None 'O']]
Enter the position to move (row,col)
Row: 0
Col: 2
[['X' None 'O']
 [None 'O' None]
 ['X' None 'O']]
[['X' None 'O']
 ['X' 'O' None]
 ['X' None 'O']]
Game Over: X wins.



=========THUẬT TOÁN CẮT TỈA ALPHA_BETA ỨNG DỤNG VỚI BÀI TOÁN TICTACTOE======
*Ý tưởng chung
Thuật toán Alpha–Beta Pruning là phiên bản tối ưu hóa của Minimax, cho phép cắt bỏ (prune) những nhánh chắc chắn không ảnh hưởng đến kết quả cuối cùng.
Minimax duyệt toàn bộ cây trạng thái → rất chậm khi cây lớn.
Alpha–Beta chỉ duyệt những nhánh cần thiết → nhanh hơn nhiều
Kết quả trả về giống Minimax, nhưng thời gian tính toán giảm mạnh.
Thuật toán hoạt động dựa trên 2 ngưỡng:
Alpha (α): giá trị tốt nhất mà MAX đã có (lớn nhất).
Beta (β): giá trị tốt nhất mà MIN đã có (nhỏ nhất).
Khi α ≥ β, cắt ngang — vì MAX sẽ không chọn nhánh tệ hơn giá MIN đã có, và ngược lại.
